In [ ]:
import os
import numpy as np
from iqtools import plotters, tools
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
%matplotlib inline
import iqtools as iq

Load the data

In [ ]:
#file = "/Users/carloforconi/Desktop/PhD/coding/results/Mo/files/98Mo_processed_spectrograms_5kHz_crop.npz"
file = "/Users/carloforconi/Desktop/PhD/coding/scripts/AN/files/98Mo_9.npz"
loaded_data = np.load(file, allow_pickle=True)

f_axis = loaded_data["f_axis"]
t_axis = loaded_data["t_axis"]
summed_specs = loaded_data["summed_specs"]
filenames = loaded_data["filenames"]

Subtract the baseline from the data

In [ ]:
'''
import sys
sys.path.append("/Users/carloforconi/Desktop/PhD/coding/git/baseline-estimate")
from nonparams_est import NONPARAMS_EST

for i in range(len(summed_specs)):
    for j in range(len(summed_specs[0,i])):
        baseline_estimator = NONPARAMS_EST(summed_specs[i][:, j])
        b_data = baseline_estimator.pls(method='arPLS', l=1e4, ratio=1e-6)
        summed_specs[i][:, j] = summed_specs[i][:, j] - b_data
'''

Plot of one random spectrogram

In [ ]:
spectrum = np.sum(summed_specs[-1], axis=0)
idx_gs = np.argmax(spectrum)
df = f_axis[0][1] - f_axis[0][0]
offset_hz = 1800   # adjust if needed
offset_bins = int(offset_hz / abs(df))
idx_iso_center = idx_gs - offset_bins
half_width = 5
idx_1 = idx_iso_center - half_width
idx_2 = idx_iso_center + half_width
slice_i = slice(idx_1, idx_2)
bkg_width = half_width*2
slice_bkg = slice(idx_1 - bkg_width, idx_1)
slice_show = slice(idx_iso_center - 30, idx_iso_center + 30)

In [ ]:
fig2, ax = plt.subplots(figsize=(8, 6))  # adjust width and height as needed

plotters.plot_spectrogram(f_axis[:,slice_show], t_axis[:,slice_show], summed_specs[-1][:,slice_show], 
                cen=0.0,      
                cmap="jet",     # colour scheme
                dpi=300,        # png resolution
                dbm=False,      # display in dBm scale
                filename=None,  # if None, no file is produced
                zzmin=1000,
                zzmax=20000,
                mask=False,     # mask out entries below this value
                decimal_place=2
                #title=f"{name} \n lframes={lframes}, nframes={nframes} \n freq corr sum"
                )
                
#ax.set_xticks(ax.get_xticks()[::1])  # Show every 1 tick
plt.xticks(rotation=45)  # Rotate labels for better readability
ax.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f"{x/1e0:.1f} Hz"))
plt.vlines(x = f_axis[0][idx_1], ymin=0, ymax=t_axis[-1][0], color='red')
plt.vlines(x = f_axis[0][idx_2], ymin=0, ymax=t_axis[-1][0], color='red')

plt.show()

Decays detection function

In [ ]:
import numpy as np
from scipy.signal import savgol_filter

def log_reject(t, step, noise, local_scale, m1, m2, reason):
    print(f"[REJECT] t={t:.6f}  "
          f"step={step:.3e}  "
          f"noise={noise:.3e}  "
          f"local_scale={local_scale:.3e}  "
          f"max={max(abs(m1), abs(m2)):.3e}  "
          f"reason={reason}")

def log_accept(t, step, noise, local_scale, m1, m2):
    print(f"[ACCEPT] t={t:.6f}  "
          f"step={step:.3e}  "
          f"noise={noise:.3e}  "
          f"local_scale={local_scale:.3e}  "
          f"max={max(abs(m1), abs(m2)):.3e}")

# main function that detects the ion decays and also rejects false decays.
def detect_all_ion_decays(time, f_axis, y_sum, y, slice_i, slice_bkg, 
                          min_points=5,
                          min_step_factor=1.0,
                          use_smoothing=True,
                          smooth_window=7,
                          require_decrease=False,
                          min_rel_step=None,
                          min_drop=0.2,
                          local_factor=0.49,
                          max_factor=0.29
                          ):

    time = np.asarray(time)
    y_sum = np.asarray(y_sum)
    n = len(y_sum)

    # ---------------------------------------------------------
    # 1. Compute areas (signal + background)
    # ---------------------------------------------------------
    A_i_list = []
    A_bkg_list = []

    freq_slice = f_axis[slice_i]
    freq_bkg   = f_axis[slice_bkg]

    order     = np.argsort(freq_slice)
    order_bkg = np.argsort(freq_bkg)

    for i in range(n):
        y_slice = y[slice_i, i][order]
        A_i = np.trapezoid(y_slice, freq_slice[order])
        A_i_list.append(A_i)

        y_slice_bkg = y[slice_bkg, i][order_bkg]
        A_bkg = np.trapezoid(y_slice_bkg, freq_bkg[order_bkg])
        A_bkg_list.append(A_bkg)

    A_i_list   = np.array(A_i_list)
    A_bkg_list = np.array(A_bkg_list)

    base_trace = A_i_list - A_bkg_list

    # ---------------------------------------------------------
    # 2. Smoothing for segmentation
    # ---------------------------------------------------------
    if use_smoothing:
        w = max(3, int(smooth_window))
        if w % 2 == 0:
            w += 1
        y_seg = savgol_filter(base_trace, window_length=w, polyorder=2)
    else:
        y_seg = base_trace

    # ---------------------------------------------------------
    # 3. REWRITTEN SEGMENTATION (THIS IS THE FIX)
    # ---------------------------------------------------------
    def split_segment(start, end):
        length = end - start
        if length < 2 * min_points:
            return [start, end]

        # ---------------------------------------------------------
        # 3.1 Find best split index by SSE minimization
        # ---------------------------------------------------------
        best_idx = None
        best_sse = np.inf

        for k in range(start + min_points, end - min_points):
            left  = y_seg[start:k]
            right = y_seg[k:end]

            m1 = left.mean()
            m2 = right.mean()

            sse = ((left - m1)**2).sum() + ((right - m2)**2).sum()

            if sse < best_sse:
                best_sse = sse
                best_idx = k

        if best_idx is None:
            return [start, end]

        # ---------------------------------------------------------
        # 3.2 Evaluate the step on UNSMOOTHED base_trace
        # ---------------------------------------------------------
        left  = base_trace[start:best_idx]
        right = base_trace[best_idx:end]

        m1 = left.mean()
        m2 = right.mean()
        step = abs(m1 - m2)

        # Robust noise estimate
        def robust_std(x):
            med = np.median(x)
            mad = np.median(np.abs(x - med))
            return 1.4826 * mad

        noise = max(robust_std(left), robust_std(right))
        if noise == 0:
            return [start, end]

        # --- HYBRID LOCAL SCALE FIX ---
        segment_trace = base_trace[start:end]

        # local estimate
        if len(segment_trace) > 2:
            local_seg_scale = np.median(np.abs(np.diff(segment_trace)))
        else:
            local_seg_scale = 0

        # global estimate
        global_scale = np.median(np.abs(np.diff(base_trace)))

        # hybrid scale: local, but never too small
        local_scale = max(local_seg_scale, 0.60 * global_scale)
        # --- END FIX ---


        # ---------------------------------------------------------
        # 3.3 REJECTION CONDITIONS — STOP RECURSION COMPLETELY
        # ---------------------------------------------------------

        t_split = time[best_idx]

        if require_decrease and not (m2 < m1):
            log_reject(t_split, step, noise, local_scale, m1, m2, "no decrease")
            return [start, end]

        if step < local_factor * local_scale:
            print("step: ", step)
            print("cost*local scale: ",local_factor * local_scale)
            log_reject(t_split, step, noise, local_scale, m1 , m2, f"step < {local_factor} * local_scale")
            return [start, end]

        if step < max_factor * max(abs(m1), abs(m2)):
            print("step: ", step)
            print("cost*max(abs(m1), abs(m2)): ", max_factor * max(abs(m1), abs(m2)))
            log_reject(t_split, step, noise, local_scale, m1, m2, f"step < {max_factor} * max plateau")
            return [start, end]

        if step < min_step_factor * noise:
            print("step: ", step)
            print("min_step_factor * noise: ", min_step_factor * noise)
            log_reject(t_split, step, noise, local_scale, m1, m2, "step < noise threshold")
            return [start, end]

        # ---------------------------------------------------------
        # 3.4 ACCEPTED SPLIT → recurse
        # ---------------------------------------------------------
        log_accept(t_split, step, noise, local_scale, m1, m2)
        left_bounds  = split_segment(start, best_idx)
        right_bounds = split_segment(best_idx, end)

        return left_bounds[:-1] + right_bounds

    # Run segmentation
    boundaries = sorted(set(split_segment(0, n)))
    # ---------------------------------------------------------
    # 4. Compute stats
    # ---------------------------------------------------------
    stats = []
    for i in range(len(boundaries) - 1):
        s = boundaries[i]
        e = boundaries[i + 1]

        stats.append({
            "t_start": time[s],
            "t_end":   time[e - 1],
            "mean_y":  y_sum[s:e].mean(),
            "std_y":   y_sum[s:e].std(ddof=1),
            "mean_area":      A_i_list[s:e].mean(),
            "mean_area_bkg":  A_bkg_list[s:e].mean()
        })

    # ---------------------------------------------------------
    # 5. GLOBAL AREA-DROP FILTER (remove fake decays)
    # ---------------------------------------------------------
    if len(stats) > 1:
        area_drops = [abs(stats[i+1]["mean_area"] - stats[i]["mean_area"])
                    for i in range(len(stats)-1)]

        median_drop = np.median(area_drops)
        MIN_DROP = min_drop * median_drop
        print("min drop: ", MIN_DROP)

        new_stats = [stats[0]]
        new_boundaries = [boundaries[0]]   # start boundary only

        for i in range(1, len(stats)):
            drop = abs(stats[i]["mean_area"] - stats[i-1]["mean_area"])

            # actual decay time between plateau i-1 and i
            decay_idx = boundaries[i]
            decay_time = time[decay_idx]

            print(f"t = {decay_time:.6f} drop: {drop}")

            if drop >= MIN_DROP:
                # keep plateau i
                new_stats.append(stats[i])
                new_boundaries.append(boundaries[i])   # <-- FIXED
            else:
                print(f"level dropped at decay time ≈ {decay_time:.6f} s")

        # always append final boundary
        new_boundaries.append(boundaries[-1])

        stats = new_stats
        boundaries = new_boundaries



    return boundaries, stats, A_i_list, A_bkg_list


In [ ]:
import matplotlib.pyplot as plt
from scipy.signal import savgol_filter
import numpy as np

def compare_smoothing_windows(t, y, windows, decay_times, polyorder=2):
    """
    Plot raw data and several smoothed versions to tune smoothing window.
    
    Parameters
    ----------
    t : array
        Time axis
    y : array
        Raw intensity trace
    windows : list of ints
        Window lengths to test (must be odd)
    polyorder : int
        Polynomial order for Savitzky-Golay
    """

    plt.figure(figsize=(12, 6))
    plt.plot(t, np.log(y), color='black', alpha=0.4, label='Raw data')

    for w in windows:
        if w % 2 == 0:
            w += 1  # enforce odd window
        y_smooth = savgol_filter(y, window_length=w, polyorder=polyorder)
        plt.plot(t, np.log(y_smooth), linewidth=2, label=f"SG window={w}")

    for t in decay_times:
        plt.vlines(t, ymin=np.min(np.log(y_smooth)), ymax=np.max(np.log(y_smooth)), color='red', linestyle='--', linewidth=1)


    plt.xlabel("Time [s]")
    plt.ylabel("Intensity [PSD]")
    plt.title("Comparison of smoothing windows")
    plt.legend()
    plt.grid(alpha=0.3)
    plt.show()


From the second run on, it is possible to rerun the code with the previous value of area found for each platau, to enhance precision.

In [ ]:
# First Run
USE_AREA_CONSISTENCY = False
A_means_old = None

In [ ]:
# Second Run 
USE_AREA_CONSISTENCY = True
A_means_old = np.load("A_means.npy", allow_pickle=True).item()

Ion number classifier: finds the number of ions for each plateau, and performs a check on lower plateaus.

In [ ]:
def classify_plateaus_hybrid(stats, boundaries, background_areas, A_means=None, k_sigma=7):
    """
    Carlo's physically-correct classifier:

    - If A_means exists: lowest plateau assigned to whichever cloud (0 or 1) it is closest to.
    - If A_means is None: fallback to background-only classification.
    - Higher plateaus assigned monotonically upward.
    - Plateau 1 merged into plateau 0 only when A_means exists (needs A0_std).
    - Monotonicity always preserved.
    """

    means = np.array([s["mean_area"] for s in stats])
    N = len(means)
    if N == 0:
        return [], stats, boundaries

    # -------------------------------
    # Background stats
    # -------------------------------
    bg_mean = np.mean(background_areas)
    bg_std  = np.std(background_areas, ddof=1) if len(background_areas) > 1 else 0.0
    if np.isnan(bg_std):
        bg_std = 0.0

    # -------------------------------
    # Step 1: classify lowest plateau
    # -------------------------------
    ion_counts = [None] * N
    A_low = means[-1]

    # A_means is usable only if it contains valid 0 and 1 distributions
    have_Ameans = (
        A_means is not None and
        0 in A_means and len(A_means[0]) > 0 and
        1 in A_means and len(A_means[1]) > 0
    )

    if have_Ameans:
        # Use learned distributions
        A0_mean = np.mean(A_means[0]); A0_std = np.std(A_means[0])
        A1_mean = np.mean(A_means[1]); A1_std = np.std(A_means[1])

        d0 = abs(A_low - A0_mean)
        d1 = abs(A_low - A1_mean)

        n_lowest = 0 if d0 <= d1 else 1

    else:
        # First run: background-only classification
        if A_low < bg_mean + k_sigma * bg_std:
            n_lowest = 0
        else:
            n_lowest = 1

    print("A_low: ", A_low)
    print("bg_mean + k_sigma * bg_std: ", bg_mean + k_sigma * bg_std)

    ion_counts[-1] = n_lowest

    # -------------------------------
    # Step 2: assign higher plateaus monotonically upward
    # -------------------------------
    for i in range(N-2, -1, -1):
        ion_counts[i] = ion_counts[i+1] + 1

    # -------------------------------
    # Step 3: merge plateau 1 into plateau 0 (only if A_means exists)
    # -------------------------------
    if N >= 2 and have_Ameans:
        A0 = means[-1]      # lowest plateau
        A1 = means[-2]      # next plateau
        A0_std = np.std(A_means[0])

        # plateau 1 is also close to 0-ion cloud
        if abs(A1 - A0) < 1.1 * A0_std:
            print("plateau merged")
            print("abs(A1 - A0): ", abs(A1 - A0))
            print(" 1.2 * A0_std: ",  1.2 * A0_std)
            ion_counts[-2] = 0

            # merge plateau 1 and plateau 0
            stats[-2]["t_end"] = stats[-1]["t_end"]
            boundaries[-2] = boundaries[-1]

            # remove last plateau
            stats = stats[:-1]
            boundaries = boundaries[:-1]
            ion_counts = ion_counts[:-1]

            # shift all higher plateaus down by 1
            for i in range(len(ion_counts)-1):
                ion_counts[i] -= 1

    return ion_counts, stats, boundaries


In [ ]:
# Convert 2D axes to 1D
if f_axis.ndim == 2:
    f_axis = f_axis[0, :]   # first row = frequency vector

if t_axis.ndim == 2:
    t_axis = t_axis[:, 0]   # first column = time vector


Parameters to tune

In [ ]:
window_fine = 11    # window of the fine moving average
window_coarse = 17  # window of the coarse moving average
min_step_factor = 0.55   # factor for noise check
tol = 0.041 # 40 ms     # tolerance for accepting values from fine and coarse
min_drop = 0.2     # factor for accepting values
tol_merging = 0.051     # tolerance for merging two values in coarse
k_sigma = 7
local_factor = 0.49
max_factor = 0.25

Main Script

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
import numpy as np

areas_list = []
all_area_diffs = []   
areas_bkg_list = []
decay_times_list = []
intensity_list, intensity_err_list = [], []
all_plateau_diffs = []
areas_list_final, all_area_diffs_final = [], [] 
transition_diffs = {}   # keys = (n, n-1), values = list of diffs
A_means = {}
A_vs_n_all = []
def ion_label(n): return "ion" if n == 1 else "ions"

for j, file in enumerate(filenames):
    print(j)
    y = summed_specs[j].T
    spectrum = np.sum(summed_specs[j], axis=0)
    idx_gs = np.argmax(spectrum)
    df = f_axis[1] - f_axis[0]
    offset_hz = 1800   # adjust if needed
    offset_bins = int(offset_hz / abs(df))
    idx_iso_center = idx_gs - offset_bins
    half_width = 5
    idx_1 = idx_iso_center - half_width
    idx_2 = idx_iso_center + half_width
    slice_i = slice(idx_1, idx_2)
    y_i = y[slice_i]              # shape (M, N)
    y_sum = y_i.sum(axis=0)
    bkg_width = idx_2 - idx_1
    slice_bkg = slice(idx_1 - bkg_width, idx_1)

    # For plotting
    slice_show = slice(idx_iso_center - 10, idx_iso_center + 15)

    F_show = f_axis[slice_show]          # 1D frequency slice
    T_show = t_axis                      # full time axis

    XX, YY = np.meshgrid(F_show, T_show)

    # ---------------------------------------------------------
    # Smoothing (y_smooth) + local uncertainty (y_std)
    # ---------------------------------------------------------
    #kernel = np.ones(window) / window
    #y_smooth = np.convolve(y_sum, kernel, mode='same')
    y_smooth = savgol_filter(y_sum, window_length=window_fine, polyorder=2)

    y_std = np.array([
        y_sum[max(0, i - window_fine // 2):min(len(y_sum), i + window_fine // 2 + 1)].std(ddof=1)
        / np.sqrt(window_fine)
        for i in range(len(y_sum))
    ])

    # ---------------------------------------------------------
    # Decay detection
    # ---------------------------------------------------------
    bound_fine, stats, A_i_list, A_bkg_list = detect_all_ion_decays(
        time=t_axis,
        f_axis=f_axis,
        y_sum=y_sum,
        y=y,
        slice_i=slice_i,
        slice_bkg=slice_bkg,
        min_points=7,
        min_step_factor=min_step_factor,
        use_smoothing=True,
        smooth_window=window_fine,
        require_decrease=True,
        min_drop=min_drop,
        local_factor=local_factor,
        max_factor=max_factor
        #min_rel_step=0.003 # 5% drop
    )

    means = [s["mean_area"] for s in stats]
    plateau_diffs = np.abs(np.diff(means))
    all_plateau_diffs.extend(plateau_diffs)

    # ---------------------------------------------------------
    # MULTI-SCALE DECAY VALIDATION
    # ---------------------------------------------------------

    # Coarse detection (more smoothing → fewer false positives)
    bound_coarse, _, _, _ = detect_all_ion_decays(
        time=t_axis,
        f_axis=f_axis,
        y_sum=y_sum,
        y=y,
        slice_i=slice_i,
        slice_bkg=slice_bkg,
        min_points=5,
        min_step_factor=min_step_factor,
        use_smoothing=True,
        smooth_window=window_coarse,   # coarse smoothing
        require_decrease=True,
        min_drop=min_drop,
        local_factor=local_factor,
        max_factor=max_factor
    )

    def ensure_indices(bounds):
        out = []
        for b in bounds:
            if isinstance(b, (int, np.integer)):
                out.append(b)
            else:
                # b is a time → convert to nearest index
                out.append(int(np.argmin(np.abs(t_axis - b))))
        return out

    bound_fine = ensure_indices(bound_fine)
    bound_coarse = ensure_indices(bound_coarse)


    # Convert boundaries → decay times
    def boundaries_to_times(bounds):
        return [t_axis[b] for b in bounds[1:-1]]

    decays_fine   = boundaries_to_times(bound_fine)
    print("decays_fine: ", decays_fine)
    decays_coarse = boundaries_to_times(bound_coarse)
    print("decays_coarse: ", decays_coarse)
    merged = []
    for t in decays_coarse:
        if not merged:
            merged.append(t)
        else:
            prev = merged[-1]

            if abs(t - prev) < tol_merging:

                # Check if either coarse value matches a fine decay
                matches_prev = [tf for tf in decays_fine if abs(tf - prev) < tol]
                matches_t    = [tf for tf in decays_fine if abs(tf - t)    < tol]

                if matches_prev:
                    # keep midpoint of prev and its fine match
                    merged[-1] = 0.5 * (prev + matches_prev[0])

                elif matches_t:
                    # keep midpoint of t and its fine match
                    merged[-1] = 0.5 * (t + matches_t[0])

                else:
                    # normal merge
                    merged[-1] = 0.5 * (t + prev)

            else:
                merged.append(t)

    decays_coarse = merged
    print("decays_coarse after merging:", decays_coarse)


    # Keep only decays that appear in both (within tolerance)
    final_decays = []

    used_fine = set()   # <-- NEW: track which fine decays have been used

    for tc in decays_coarse:

        # find all fine decays close to this coarse decay
        close = [(tf, abs(tf - tc)) for tf in decays_fine
                if tf not in used_fine and abs(tf - tc) < tol]

        if len(close) == 1:
            tf, _ = close[0]
            final_decays.append((tf + tc) / 2)
            used_fine.add(tf)   # <-- mark as used

        elif len(close) > 1:
            # keep only the closest fine decay
            tf, _ = min(close, key=lambda x: x[1])
            final_decays.append((tf + tc) / 2)
            used_fine.add(tf)   # <-- mark as used

    # if len(close) == 0 → skip (coarse-only decay)

    print("Decay times: ", final_decays)
    
    # Rebuild boundaries from final_decays
    boundaries = [0] + [np.argmin(abs(t_axis - t)) for t in final_decays] + [len(t_axis)-1]
    boundaries = sorted(set(boundaries))
    #print("Boundaries 1: ", boundaries)

    # ---------------------------------------------------------
    # Rebuild stats to match the NEW boundaries
    # ---------------------------------------------------------
    new_stats = []
    for i in range(len(boundaries) - 1):
        s = boundaries[i]
        e = boundaries[i + 1]

        new_stats.append({
            "t_start": t_axis[s],
            "t_end":   t_axis[e - 1],
            "mean_y":  y_sum[s:e].mean(),
            "std_y":   y_sum[s:e].std(ddof=1),
            "mean_area":      A_i_list[s:e].mean(),
            "mean_area_bkg":  A_bkg_list[s:e].mean()
        })

    stats = new_stats

    #decay_times = [t_axis[b] for b in boundaries[1:-1]]
    #decay_times_list.append(decay_times)

    # ---------------------------------------------------------
    # Area differences
    # ---------------------------------------------------------
    areas_bkg = [seg["mean_area_bkg"] for seg in stats]
    areas_bkg_list.append(areas_bkg)

    intensity = [seg["mean_y"] for seg in stats]
    intensity_err = [seg["std_y"] for seg in stats]
    intensity_list.append(intensity)
    intensity_err_list.append(intensity_err)

    ion_counts, stats, boundaries = classify_plateaus_hybrid(stats, boundaries=boundaries, background_areas=areas_bkg, A_means=A_means_old)

    areas = [seg["mean_area"] for seg in stats]
    areas_list.append(areas)
    area_diffs = [areas[i+1] - areas[i] for i in range(len(areas)-1)]
    all_area_diffs.append(area_diffs)

    # --- Save area → ion mapping for this file ---
    A_vs_n = []

    for A, n in zip(areas, ion_counts):
        A_vs_n.append((n, A))
        A_means.setdefault(n, []).append(A)

    # Store it in a global list if you want to analyze later
    A_vs_n_all.append(A_vs_n)

    # --------------------------------------------------------- 
    # # Merge consecutive plateaus with same ion count (time order) 
    # # --------------------------------------------------------- 
    #boundaries, stats, ion_counts = merge_consecutive_plateaus(boundaries, stats, ion_counts)

    # FIX: remove duplicate boundaries
    cleaned = []
    for b in boundaries:
        if not cleaned or cleaned[-1] != b:
            cleaned.append(b)
    boundaries = cleaned
    # ---------------------------------------------------------
    # Collect area differences grouped by ion transition
    # ---------------------------------------------------------
    means = [s["mean_area"] for s in stats]

    for i in range(len(stats) - 1):
        n1 = ion_counts[i]
        n2 = ion_counts[i+1]

        # Only consider real decays: ion count must decrease by 1
        if n1 - n2 == 1:
            diff = abs(means[i] - means[i+1])
            key = (n1, n2)

            if key not in transition_diffs:
                transition_diffs[key] = []

            transition_diffs[key].append(diff)

    areas_final = [seg["mean_area"] for seg in stats]
    areas_list_final.append(areas_final)
    area_diffs_final = [areas[i+1] - areas[i] for i in range(len(areas)-1)]
    all_area_diffs_final.append(area_diffs_final)
    
    # # recompute decay_times after merge 
    #decay_times = [t_axis[b] for b in boundaries[1:-1]] 
    decay_times = [t_axis[b] for b in boundaries[1:-1]]
    decay_times_list.append(decay_times)
    '''decay_times = []
    for i in range(1, len(stats)):
        if ion_counts[i] != ion_counts[i-1]:
            # transition between plateau i-1 and i
            t_left  = stats[i-1]["t_end"]
            t_right = stats[i]["t_start"]
            t_mid = 0.5 * (t_left + t_right)
            decay_times.append(t_mid)
    '''
    #print("Boundaries:", boundaries)
    #print("Decay times after merging:", decay_times)

    # ---------------------------------------------------------
    # Create side-by-side figure
    # ---------------------------------------------------------
    fig, (ax_left, ax_right) = plt.subplots(1, 2, figsize=(16, 6))

    # =========================================================
    # LEFT PANEL — Intensity over time
    # =========================================================
    ax_left.plot(t_axis, (y_smooth), linewidth=2, color='green', label='Moving average')
    ax_left.fill_between(t_axis, (y_smooth - y_std), (y_smooth + y_std),
                         color='green', alpha=0.2, label='Uncertainty')
    ax_left.plot(t_axis, (y_sum), label='Data', color='blue')
    #print("decayyyyy", decay_times)
    for t in decay_times:
        ax_left.axvline(t, color='red', linestyle='--', linewidth=1)

    legend_entries = []
    header = Line2D([], [], color='none')
    legend_entries.append((header, "Interval            |            Mean PSD             |  Mean Area"))

    for seg, ions in zip(stats, ion_counts):
        row = Line2D([], [], color='none')
        label = (f"{seg['t_start']:.2f}–{seg['t_end']:.2f} s  |  "
                f"{seg['mean_y']:.2e} ± {seg['std_y']:.2e}  |  "
                f"{seg['mean_area']:.2e}  |  {ions} ions")
        legend_entries.append((row, label))

    # annotate ions in intensity plot
    for i, seg in enumerate(stats):
        t0 = seg["t_start"]
        t1 = seg["t_end"]
        t_mid = 0.5 * (t0 + t1)

        ions = ion_counts[i]

        ax_left.text(
            t_mid,
            ax_left.get_ylim()[0],          # slightly above the curve
            f"{ions} {ion_label(ions)}",
            ha='center',
            va='bottom',
            fontsize=10,
            bbox=dict(facecolor='white', alpha=0.95, edgecolor='black')
        )

    # annotate ions in spectrogram plot
    for i, seg in enumerate(stats):
        t0 = seg["t_start"]
        t1 = seg["t_end"]
        t_mid = 0.5 * (t0 + t1)

        ions = ion_counts[i]

        ax_right.text(
            f_axis[slice_show].min(),   # left side of spectrogram
            t_mid,
            f"{ions} {ion_label(ions)}",
            ha='left',
            va='center',
            fontsize=10,
            color='white',
            bbox=dict(facecolor='black', alpha=0.95, edgecolor='white')
        )



    handles = [h for h, _ in legend_entries]
    labels = [l for _, l in legend_entries]
    ax_left.legend(handles, labels, loc='upper right', fontsize=8, frameon=True)

    ax_left.grid()
    ax_left.set_xlabel("Time [s]")
    ax_left.set_ylabel("Intensity [PSD]")
    ax_left.set_title("Isomer intensity over time with decay detection")
    # Convert area_diffs to a readable string
    area_diff_str = ", ".join([f"{-d:.2e}" for d in area_diffs])
    # Add to legend
    row = Line2D([], [], color='none')
    label = f"Area diffs: {area_diff_str}"
    legend_entries.append((row, label))
    handles = [h for h, _ in legend_entries]
    labels = [l for _, l in legend_entries]
    ax_left.legend(handles, labels, loc='upper right', fontsize=8, frameon=True)

    # =========================================================
    # RIGHT PANEL — Spectrogram
    # =========================================================
    plotters.plot_spectrogram(
        XX.T, YY.T, summed_specs[j][:,slice_show].T,
        cen=0.0,
        cmap="jet",
        dpi=300,
        dbm=False,
        filename=None,
        zzmin=1000,
        zzmax=20000,
        mask=False,
        decimal_place=2
    )

    ax_right.set_title("Spectrogram")
    ax_right.set_xticklabels(ax_right.get_xticklabels(), rotation=45)
    ax_right.xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: f"{x:.1f} Hz"))

    ax_right.vlines(x=f_axis[idx_1], ymin=0, ymax=t_axis[-1], color='red')
    ax_right.vlines(x=f_axis[idx_2], ymin=0, ymax=t_axis[-1], color='red')
    ax_right.vlines(x=f_axis[idx_gs], ymin=0, ymax=t_axis[-1], color='yellow')

    for t in decay_times: 
        ax_right.axhline(t, color='white', linestyle='--', linewidth=1)

    fig.suptitle(file.split("/")[-1], fontsize=12)
    plt.tight_layout()

    plt.figure(figsize=(8,5))
    plt.hist(y_sum, bins=60, color='steelblue', alpha=0.75, label='Data')
    plt.hist(y_smooth, bins=60, color='green', alpha=0.75, label='Moving avg')

    plt.xlabel("Intensity [PSD]")
    plt.ylabel("Counts")
    plt.title("Histogram of intensity values with ion labels")
    plt.grid(alpha=0.3)

    # Add plateau markers
    for seg, ions in zip(stats, ion_counts):
        mean_y = seg["mean_y"]
        plt.axvline(mean_y, color='red', linestyle='--', linewidth=1)

        plt.text(
            mean_y,
            plt.ylim()[1] * 0.9,     # near the top of the histogram
            f"{ions} {ion_label(ions)}",
            ha='center',
            
            va='bottom',
            fontsize=10,
            bbox=dict(facecolor='white', alpha=0.9, edgecolor='black')
        )
    #compare_smoothing_windows(t_axis, y_sum, windows=[9, 17], decay_times=decay_times)

    plt.show()


# ---------------------------------------------------------
# After the loop
# ---------------------------------------------------------
print("Area differences for each file:")
for fname, diffs in zip(filenames, all_area_diffs):
    print(fname.split("/")[-1], diffs)


In [ ]:
for ion, areas in A_means.items():
    #print("ion number: ", ion)
    #print("areas: ", areas)
    mean_A = np.mean(areas)
    std_A  = np.std(areas, ddof=1)
    print(f"Ion {ion}: mean = {mean_A:.6e}, std = {std_A:.6e}, count = {len(areas)}")


In [ ]:
#np.save("A_means.npy", A_means)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# ============================================================
# LEFT PANEL — Mean plateau area vs ion number
# ============================================================

ion_numbers = sorted(A_means.keys())
mean_areas = np.array([np.mean(A_means[n]) for n in ion_numbers])
std_areas  = np.array([np.std(A_means[n], ddof=1) for n in ion_numbers])

# Linear fit
a_lin, b_lin = np.polyfit(ion_numbers, mean_areas, deg=1)

# Quadratic fit
a2, b2, c2 = np.polyfit(ion_numbers, mean_areas, deg=2)

n_fit = np.linspace(min(ion_numbers), max(ion_numbers), 200)
A_fit_lin  = a_lin*n_fit + b_lin
A_fit_quad = a2*n_fit**2 + b2*n_fit + c2

print(f"Linear fit:     A(n) = {a_lin:.3e} n + {b_lin:.3e}")
print(f"Quadratic fit:  A(n) = {a2:.3e} n^2 + {b2:.3e} n + {c2:.3e}")

# ============================================================
# RIGHT PANEL — ΔA(n) computed exactly as you did it
# ============================================================

# Your method: simply take the diff of mean_areas
area_diffs = np.diff(mean_areas)

# x-values for the diffs: transitions 1→0, 2→1, 3→2, ...
# If ion_numbers = [0,1,2,3,4,5], then diffs correspond to n = [1,2,3,4,5]
n_vals = ion_numbers[1:]   # skip the first one

# Build labels like "1→0", "2→1", ...
labels = [f"{n}→{n-1}" for n in n_vals]

# ============================================================
# COMBINED FIGURE
# ============================================================

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14,5))

# ---------------- LEFT PANEL ----------------
ax1.errorbar(
    ion_numbers, mean_areas, yerr=std_areas,
    fmt='o', markersize=8, capsize=5, label="Mean area ± 1 std"
)
ax1.plot(n_fit, A_fit_lin,  color='green', linestyle='--', linewidth=2, label="Linear fit")
ax1.plot(n_fit, A_fit_quad, color='red',   linestyle='--', linewidth=2, label="Quadratic fit")

ax1.set_xlabel("Number of ions")
ax1.set_ylabel("Mean area")
ax1.set_title("Mean plateau area vs ion number")
ax1.grid(alpha=0.3)
ax1.legend()

# ---------------- RIGHT PANEL ----------------
ax2.plot(
    n_vals, area_diffs,
    'o-', markersize=8, label="ΔA(n) = A(n) - A(n-1)"
)

ax2.set_xticks(n_vals)
ax2.set_xticklabels(labels)

ax2.set_ylabel("Area difference")
ax2.set_xlabel("Ion transition")
ax2.set_title("Mean area difference per ion transition")
ax2.grid(alpha=0.3)
ax2.legend()

plt.tight_layout()
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

colors = ["blue", "green", "orange", "pink"]

# --- compute stats ---
stats = {}
for key in [(1,0), (2,1), (3,2), (4,3)]:
    if key in transition_diffs and len(transition_diffs[key]) > 0:
        arr = np.array(transition_diffs[key])
        arr = arr[arr < np.percentile(arr, 80)]

        stats[key] = {
            "mean": np.mean(arr),
            "var": np.var(arr, ddof=1),
            "std": np.std(arr, ddof=1),
            "n": len(arr)
        }

# --- plot histograms ---
plt.figure(figsize=(10,6))

for idx, key in enumerate([(1,0), (2,1), (3,2), (4,3)]):
    if key in transition_diffs:
        mean = stats[key]["mean"]
        std  = stats[key]["std"]
        label = f"{key[0]}→{key[1]}  (μ={mean:.2e}, σ={std:.2e})"

        plt.hist(
            transition_diffs[key],
            bins=40,
            alpha=0.4,
            color=colors[idx],
            label=label
        )

        # vertical line at the mean
        plt.axvline(
            mean,
            color=colors[idx],
            linestyle="--",
            linewidth=2
        )

plt.xlabel("Area difference")
plt.ylabel("Count")
plt.title("Area differences grouped by ion transition")
plt.legend()
plt.grid(alpha=0.3)
plt.show()


In [ ]:
print(means)
ratio = []
for i in range(len(means)-1):
    ratio.append(means[i+1]/means[i])

print(ratio)

In [ ]:
A_means = {}
for ion_count, area in zip(ion_counts, areas):
    A_means.setdefault(ion_count, []).append(area)

means = []
for n, arr in A_means.items():
    print(n, np.mean(arr))
    means.append(np.mean(arr))

ratio = []
for i in range(len(means)-1):
    ratio.append(means[i+1]/means[i])

print(ratio)

In [ ]:
plt.figure(figsize=(8,5))
flat = []
for x in (all_plateau_diffs):
    flat.append(x)
#flat = [x for diffs in np.array(all_plateau_diffs) for x in diffs]
flat = np.array(flat)
flat_sorted = sorted(flat)
plt.hist(np.array(flat_sorted)[:-2], bins=60, density=True, alpha=0.4, color='steelblue', label='Histogram')

from scipy.stats import gaussian_kde
kde = gaussian_kde(np.array(flat_sorted)[:-2])
xs = np.linspace(min(np.array(flat_sorted)[:-2]), max(np.array(flat_sorted)[:-2]), 500)
#plt.plot(xs, kde(xs), color='red', linewidth=2, label='KDE')

plt.xlabel("Area difference")
plt.ylabel("Density")
plt.grid(alpha=0.3)
plt.legend()
plt.title("Distribution of area differences across all files")
plt.show()

from sklearn.cluster import KMeans

X = np.array(flat_sorted)[:-2].reshape(-1,1)
kmeans = KMeans(n_clusters=4).fit(X)
centers = sorted(kmeans.cluster_centers_.flatten())

print("Cluster centers:", centers)



In [ ]:
from scipy.signal import savgol_filter
%matplotlib inline
plateau_means = []
step_starts = [] 
step_ends = []

y = np.array(sorted(flat)[::-1])
x = np.arange(len(y))

y_smooth = savgol_filter(y, window_length=11, polyorder=2)
dy = np.diff(y)

noise = np.median(np.abs(dy - np.median(dy))) * 1.4826
neg_threshold = -1.5 * noise
flat_threshold = 1 * noise

i = 0
while i < len(dy):
    # detect start of a downward step
    if dy[i] < neg_threshold:
        step_starts.append(i)

        # find where derivative returns to near zero
        j = i + 1
        while j < len(dy) and abs(dy[j]) > flat_threshold:
            j += 1

        step_ends.append(j)
        i = j
    else:
        i += 1

boundaries = [0]
for s, e in zip(step_starts, step_ends):
    boundaries.append(s)   # plateau ends before step
    boundaries.append(e)   # next plateau starts after step
boundaries.append(len(y))

plateau_means = []
for i in range(0, len(boundaries)-1, 2):
    s = boundaries[i]
    e = boundaries[i+1]
    plateau_means.append(y[s:e].mean())

step_sizes = np.diff(plateau_means)[1:]

plt.figure(figsize=(10,6))
plt.plot(x, y, label="Data")
#plt.plot(x, y_smooth, label="Smoothed", alpha=0.7)

# plot plateaus
for i in range(0, len(boundaries)-1, 2):
    s = boundaries[i]
    e = boundaries[i+1]
    plt.hlines(plateau_means[i//2], s, e, colors='red', linewidth=2, linestyle="--")

# plot step transitions
for s, e in zip(step_starts, step_ends):
    plt.axvline(s, color='green', linestyle='--', alpha=0.3)
    plt.axvline(e, color='green', linestyle='--', alpha=0.3)

#plt.gca().invert_xaxis()
plt.xlabel("Number of Particles")
plt.ylabel("Area Values")
plt.title("Plateaus and Step Transitions")
plt.grid()
plt.legend()
plt.show()


In [ ]:
flat = [t for sub in decay_times_list for t in sub]
t = np.asarray(flat)
lam = 1 / t.mean()
half_life = np.log(2) * t.mean()
half_life_err = half_life / np.sqrt(len(t))
x = np.linspace(0, max(flat), 500)
plt.plot(x, lam * np.exp(-lam * x))

bins = int(np.sqrt(len(flat)))  # or Freedman–Diaconis
plt.hist(flat, bins=bins, density=True, alpha=0.5, label=f"Half-life= {half_life*1000:.2f} ± {half_life_err*1000:.2f} ms")
plt.xlabel("Decay time [s]")
plt.ylabel("Counts")
plt.title("Histogram of decay times")
plt.legend()
plt.grid()
plt.show()